In [1]:
import torch
import torch.nn.functional as F
from torchvision import datasets, transforms, models
import os
import shutil
from collections import OrderedDict

In [2]:
os.getcwd()

'/home/saileshg/sailspace/dev2019/pytorch_udacity/04_intro_to_pytorch'

In [3]:
def organize_images_by_class(path, class_strings=['cat', 'dog']):
    files_in_dir = os.listdir(path)
    print(f'files_in_dir:{files_in_dir[0]}, length:{len(files_in_dir)}')
    
    for class_label in class_strings:
        cpath = os.path.join(path,class_label)
        if not os.path.isdir(cpath):
            print(f'Creating directory {class_label}')
            os.mkdir(cpath)
        
        for file_name in files_in_dir:
            src = os.path.join(path,file_name)
            if os.path.isfile(src) and class_label in file_name:
                dst = os.path.join(cpath,file_name)
                shutil.move(src,dst)
                
#     print(os.walk(path))

In [4]:
data_dir = '../../Kaggle_Learn/data/dogs-vs-cats'

organize_images_by_class(data_dir+'/train')

files_in_dir:dog, length:2


In [5]:

train_transforms = transforms.Compose([transforms.RandomRotation(30),
                                       transforms.RandomResizedCrop(224),
                                       transforms.RandomHorizontalFlip(),
                                       transforms.ToTensor(),
                                       transforms.Normalize([0.485, 0.456, 0.406],
                                                            [0.229, 0.224, 0.225])])

test_transforms = transforms.Compose([transforms.Resize(255),
                                      transforms.CenterCrop(224),
                                      transforms.ToTensor(),
                                      transforms.Normalize([0.485, 0.456, 0.406],
                                                           [0.229, 0.224, 0.225])])

train_data = datasets.ImageFolder(data_dir+'/train', transform=train_transforms)

trainloader = torch.utils.data.DataLoader(train_data, batch_size=32, shuffle=True)

In [6]:
model = models.densenet121(pretrained=True)

for param in model.parameters():
    param.requires_grad = False

Downloading: "https://download.pytorch.org/models/densenet121-a639ec97.pth" to /home/saileshg/.cache/torch/checkpoints/densenet121-a639ec97.pth
100%|██████████| 30.8M/30.8M [00:08<00:00, 3.74MB/s]


In [8]:
model

DenseNet(
  (features): Sequential(
    (conv0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (norm0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu0): ReLU(inplace=True)
    (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (denseblock1): _DenseBlock(
      (denselayer1): _DenseLayer(
        (norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplace=True)
        (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (norm2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu2): ReLU(inplace=True)
        (conv2): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
      (denselayer2): _DenseLayer(
        (norm1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu

In [9]:
layers = OrderedDict([('fc1', torch.nn.Linear(1024, 500)),
                     ('relu', torch.nn.ReLU()),
                     ('fc2', torch.nn.Linear(500, 2)),
                     ('output', torch.nn.LogSoftmax(dim=1))])

classifier = torch.nn.Sequential(layers)

# Final Layer of DenseNet121 is named: classifier
model.classifier = classifier

In [11]:
# Use GPU if it's available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

criterion = torch.nn.NLLLoss()
optimizer = torch.optim.Adam(model.classifier.parameters(), lr=0.003)

model.to(device); # `;` prevents console logging

In [12]:
num_epochs = 5

running_loss = 0
model.train()

for epoch in range(num_epochs):
    
    for inputs, labels in trainloader:
        inputs = inputs.to(device)
        labels = labels.to(device)
        
        optimizer.zero_grad()
        
        preds = model.forward(inputs)
        loss = criterion(preds, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        
    print(f'Epoch: {epoch} | Training loss: {running_loss/len(trainloader)}')
    
# validation -- pending

KeyboardInterrupt: 